In [1]:
from langchain.tools import tool
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from typing import Annotated
from langchain_core.tools import InjectedToolArg
from dotenv import load_dotenv

d:\LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

Without Annotated[float, InjectedToolArg]:

The LLM might try to “guess” a conversion rate based on its training data.

That could be completely wrong, especially if the rate has changed since the model was trained.

With InjectedToolArg:

The tool system enforces that the value comes from a reliable source, e.g., your API call.

Your function becomes deterministic: it always uses the actual rate.

In [3]:
# tool create
@tool

def get_conversion_factor(base_currency : str, target_currency : str) -> float:
    """
    This function fetches the currency coversion factorbetween a given base currency and a target currency
    
    """
    
    url = f"https://v6.exchangerate-api.com/v6/051cb422ab1f900f09447b8b/latest/{base_currency}"
    response = requests.get(url)
    data = response.json()
    
    # Return only the conversion factor for the target currency
    return data["conversion_rates"][target_currency]

@tool

#  we use annotated tool bcz of without it the llm decide the conversion_rate.so use of this annotatted tool the user/programer decide the conversion_rate.
# it means if we not use annotated the llm decide the conversion_rate vcalue acc. to the llm training data.

def convert(base_currency_value : int, conversion_rate : Annotated[float, InjectedToolArg]) -> float:
    """Given a currency conversion rate this function calculated the target a given currecny value"""
    
    return base_currency_value * conversion_rate

In [4]:
print(get_conversion_factor.name)
print(get_conversion_factor.args)
print(get_conversion_factor.description)

get_conversion_factor
{'base_currency': {'title': 'Base Currency', 'type': 'string'}, 'target_currency': {'title': 'Target Currency', 'type': 'string'}}
This function fetches the currency coversion factorbetween a given base currency and a target currency


In [5]:
get_conversion_factor.invoke({'base_currency': 'USD', 'target_currency':'INR'})

88.2896

In [6]:
convert.invoke({'base_currency_value':10, 'conversion_rate':88.7311})

887.3109999999999

In [7]:
llm = ChatGoogleGenerativeAI(model = 'gemini-2.5-pro')

In [8]:
llm_with_tool = llm.bind_tools([get_conversion_factor, convert])

In [9]:
message = [HumanMessage('what is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR')]

In [10]:
message

[HumanMessage(content='what is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [11]:
ai_message = llm_with_tool.invoke(message)

In [12]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--36039056-859a-4c72-a472-de9517462176-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '478c6ed7-840c-466e-930b-e5aa48f6400a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 148, 'output_tokens': 26, 'total_tokens': 887, 'input_token_details': {'cache_read': 0}})

In [13]:
message.append(ai_message)

In [14]:
message

[HumanMessage(content='what is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--36039056-859a-4c72-a472-de9517462176-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '478c6ed7-840c-466e-930b-e5aa48f6400a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 148, 'output_tokens': 26, 'total_tokens': 887, 'input_token_details': {'cache_read': 0}})]

In [15]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '478c6ed7-840c-466e-930b-e5aa48f6400a',
  'type': 'tool_call'}]

In [16]:
import json


for tool_call in ai_message.tool_calls:
    print(tool_call)

{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '478c6ed7-840c-466e-930b-e5aa48f6400a', 'type': 'tool_call'}


In [17]:
# execute the 1st tool and get the value of conversion rate
if tool_call['name'] == 'get_conversion_factor':
    tool_message_1 = get_conversion_factor.invoke(tool_call)
    print(tool_message_1)
    
    # fetch this conversion rate
    # conversion_rate = json.loads(tool_message_1.content)['conversion_rate']
    # print(conversion_rate)
    
    
    # append this tool message to message lis

content='88.2896' name='get_conversion_factor' tool_call_id='478c6ed7-840c-466e-930b-e5aa48f6400a'
